In [73]:
import pandas as pd
import itertools as it
import string
from collections import Counter
import json
import re
import numpy

In [48]:
with open('data/stopwords.txt') as fh:
    stopwords = set(fh.read().splitlines())

In [49]:
def normalise(text):
    
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", text)
    text = text.replace("/", " / ")
    text = re.sub(r"@\w+", " <user> ", text)
    text = re.sub(r"[8:=;]['`\-][)d]+|[)d]+['`\-][8:=;]", " <smile> ", text)
    text = re.sub(r"[8:=;]['`\-]p+", " <lolface> ", text)
    text = re.sub(r"[8:=;]['`\-]\(+|\)+['`\-][8:=;]", " <sadface> ", text)
    text = re.sub(r"[8:=;]['`\-][\/|l*]", " <neutralface> ", text)
    text = text.replace(r"<3", " <heart> ")
    text = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", text)
    text = text.replace(r"#", " <hashtag> ")
    text = re.sub(r"([!?.,()])+", r" \1 ", text)
    
    return text

In [50]:
def tokenise(text):
    return [t.strip() for t in text.strip().split()]

In [51]:
def parse_treatment_definitons(definion_file):
    treatment_set = set()
    treatment_mapping = {}
    max_length = 1
    for line in definion_file:
        line = line.strip()
        treatments = line.split(',')
        name = treatments[0]
        for treatment in treatments:
            treatment = tuple(tokenise(normalise(treatment)))
            max_length = max(len(treatment), max_length)
            treatment_set.add(treatment)
            treatment_mapping[treatment] = name
    return treatment_set, treatment_mapping, max_length

In [52]:
def window_sliding(iterable, n):
    gens = (
        it.chain(it.repeat(None, n - 1 - i), iterable, it.repeat(None, i))
        for i, gen in enumerate(it.tee(iterable, n)))
    return zip(*gens)

def find_treatments(tokens):
    found_treatments = []
    for x in range(max_treatment_length, 0, -1):
        for window in window_sliding(tokens, x):
            if tuple(window) in treatment_set:
                found_treatments.append(treatment_mapping[tuple(window)])
    return list(set(found_treatments)) or None

In [53]:
def filter_stopwords(tokens):
    return [t for t in tokens if len(t) > 2 and t not in stopwords]

In [54]:
def calculate_embeddings(tokens):
    token_embdgs = [embeddings.get(t) for t in tokens]
    token_embdgs = [t for t in token_embdgs if t is not None]
    return numpy.mean(token_embdgs, axis=0)

In [55]:
with open('data/merged_tweets.jsonl', 'r') as fh:
    df = pandas.read_json(fh.read(), lines=True, convert_dates=True)

In [56]:
df = df.loc[(df['retweeted_status'].isnull()) & (df['lang'] == "en")]

In [57]:
df = df.loc[(df.created_at >= "2017-12-04") & (df.created_at <= "2018-02-03")]

In [58]:
df["extended_tweet"] = df["extended_tweet"].apply(lambda x: x["full_text"]  if type(x) == dict else x )

In [59]:
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
11631,NaN,None,2017-12-04 00:00:52,"[13, 140]","{u'user_mentions': [{u'indices': [0, 12], u'id...",NaN,@germanBruin I watched a Ted talk about a blac...,0,False,low,...,0.0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@germanBruin I watched a Ted talk about a blac...,2017-12-04 00:00:52.006,True,"{u'follow_request_sent': None, u'profile_use_b...",NaN
11636,NaN,None,2017-12-04 00:09:44,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,0.0,0,False,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",😻😻😻Neither will I or loads of others. If THIS ...,2017-12-04 00:09:44.732,False,"{u'follow_request_sent': None, u'profile_use_b...",NaN
11645,NaN,None,2017-12-04 00:21:11,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,0.0,0,False,NaN,"<a href=""http://www.forextraining.com"" rel=""no...",#Tinnitus can be perceived in one or both ears...,2017-12-04 00:21:11.003,False,"{u'follow_request_sent': None, u'profile_use_b...",NaN
11647,NaN,None,2017-12-04 00:25:03,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,0.0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",And now i just sit in silence...\n\nJust kiddi...,2017-12-04 00:25:03.859,False,"{u'follow_request_sent': None, u'profile_use_b...",NaN
11648,NaN,None,2017-12-04 00:25:15,NaN,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,NaN,0,False,low,...,0.0,0,False,NaN,"<a href=""http://bufferapp.com"" rel=""nofollow"">...",Have a weird half tinnitus half ears want to p...,2017-12-04 00:25:15.339,False,"{u'follow_request_sent': None, u'profile_use_b...",NaN


In [60]:
df["text"] = df["extended_tweet"].combine_first(df["text"])

In [61]:
df["text"] = df["text"].apply(lambda x: x.replace("\n", " "))

In [62]:
df["user_name"] = df["user"].apply(lambda x: x["screen_name"])

In [63]:
df = df.loc[:, ["id", "created_at", "text", "retweet_count", "user_name"]]

In [64]:
df["tokens"] = df["text"].apply(lambda x: tokenise(normalise(x)))

In [65]:
len(df.user_name.unique())

7133

In [ ]:
treatment_set, treatment_mapping, max_treatment_length = parse_treatment_definitons(open("data/treatment_definitons.txt", 'r'))

In [66]:
df['filtered_tokens'] = df['tokens'].apply(filter_stopwords)

In [ ]:
word_counts = Counter()
for elem in df["filtered_tokens"]:
    word_counts += Counter(elem)

In [ ]:
word_counts.most_common(1)

In [67]:
df['treatments'] = df['tokens'].apply(find_treatments)

In [68]:
df = df.loc[df['treatments'].notnull()]

In [69]:
len(df.user_name.unique())

838

In [ ]:
df.to_csv("tweets_with_treatments.csv")

In [ ]:
treatment_counts = Counter()
for elem in df["treatments"]:
    treatment_counts += Counter(elem)

In [102]:
new_df_twitter=pd.DataFrame(treatment_counts.most_common(100))

In [112]:
new_df_twitter.columns= ['keyword','count']

In [115]:
new_df_twitter

,keyword,count
0,medicine,906
1,Sound Therapy,146
2,Masking,90
3,Cannabis,56
4,Transcranial Magnetic Stimulation,39
5,Ginkgo,27
6,Tinnintus Retraining Therapy,26
7,Cognitive Behavioural Therapy,26
8,Acupuncture,25
9,Magnesium,18


In [110]:
new_df_twitter.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
0    25 non-null object
1    25 non-null int64
dtypes: int64(1), object(1)
memory usage: 472.0+ bytes


In [79]:
new_df_twitter.to_pickle('Twitter_counts.pkl') 

In [81]:
new_df_TT = pd.read_pickle('TT_counts.pkl')

In [117]:
new_df_TT.columns= ['keyword','count']
new_df_TT

,keyword,count
0,medicine,1408
1,Steroids,401
2,Antidepressants,393
3,Tinnintus Retraining Therapy,296
4,Masking,271
5,Magnesium,189
6,Hyperbaric Oxygen Therapy,159
7,Low-level Laser Therapy,137
8,Clonazepam,136
9,Cognitive Behavioural Therapy,123


In [118]:
pd.concat([new_df_TT, new_df_twitter], axis=1)

,keyword,count,keyword,count
0,medicine,1408,medicine,906.0
1,Steroids,401,Sound Therapy,146.0
2,Antidepressants,393,Masking,90.0
3,Tinnintus Retraining Therapy,296,Cannabis,56.0
4,Masking,271,Transcranial Magnetic Stimulation,39.0
5,Magnesium,189,Ginkgo,27.0
6,Hyperbaric Oxygen Therapy,159,Tinnintus Retraining Therapy,26.0
7,Low-level Laser Therapy,137,Cognitive Behavioural Therapy,26.0
8,Clonazepam,136,Acupuncture,25.0
9,Cognitive Behavioural Therapy,123,Magnesium,18.0


In [119]:
pd.merge(new_df_TT, new_df_twitter, how='inner', on='keyword')

,keyword,count_x,count_y
0,medicine,1408,906
1,Steroids,401,4
2,Antidepressants,393,14
3,Tinnintus Retraining Therapy,296,26
4,Masking,271,90
5,Magnesium,189,18
6,Hyperbaric Oxygen Therapy,159,13
7,Low-level Laser Therapy,137,1
8,Cognitive Behavioural Therapy,123,26
9,Melatonin,120,1
